# Google Vision API

This notebook applies the google vision API to Iens restaurant pictures to detect what food is on it.

In [ ]:
import pandas as pd
import pandas_gbq as gbq 
import json
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# project specifics
PRIVATE_KEY = '../google-credentials/gsdk-credentials.json'
PROJECT_ID = json.load(open(PRIVATE_KEY))['project_id']
APIKEY = open('../google-credentials/gc-API-key.txt').read()

In [ ]:
# dataset specifics
city = 'dongen'
date = '20180124'
bq_table = '_'.join(['iens.iens', city, date])  # use iens.iens_comments when querying on the comments table

In [ ]:
# select all info fields, plus image_urls
query = "SELECT info.id, info.name, image_urls FROM {} WHERE info.nr_images > 0".format(bq_table)

df = gbq.read_gbq(query, project_id=PROJECT_ID, private_key=PRIVATE_KEY)

## Calling the vision API 

First practice - just run a single request. See if it works!

Usefull documentation: 
* https://developers.google.com/api-client-library/python/start/get_started
* https://github.com/GoogleCloudPlatform/cloud-vision/tree/master/python/text

In [ ]:
from googleapiclient.discovery import build
service = build('vision', 'v1', developerKey=APIKEY)
collection = service.images()

In [ ]:
def make_request(url):
    return {
        'image' : {
            'source' : {
                'imageUri' : url
            }
        },
        'features': [{
                    'type': 'LABEL_DETECTION',
                    'maxResults': 10
        }]
    }

def make_batch_request(url_list):
    return collection.annotate(body={'requests' : [make_request(url) for url in url_list]})

In [ ]:
examples = {'burger' : 'https://u.tfstatic.com/restaurant_photos/811/352811/169/612/barasti-killer-burger-b42ea.jpg',
            'steak' : 'https://u.tfstatic.com/restaurant_photos/811/352811/169/612/barasti-ribstuk-2c5f9.jpg'}

In [ ]:
from IPython.display import Image
Image(url=examples['burger']) 

In [ ]:
response = make_batch_request([examples['burger']]).execute()
pd.DataFrame(response['responses'][0]['labelAnnotations']).head()

## Set up batch request per restaurant

In [ ]:
restaurants_image_lists = df.groupby(['info_id', 'info_name'])['image_urls'].apply(list)

In [ ]:
restaurants_image_lists

In [ ]:
restaurants_image_lists[0]

In [ ]:
def do_batch_request(url_list):
    batch_request = make_batch_request(url_list)
    try:
        responses = batch_request.execute(num_retries=5)
        if 'responses' not in responses:
            return {}
        text_response = {}
        for image_url, response in zip(url_list, responses['responses']):
            if 'error' in response:
                print("API Error for %s: %s" % (
                        image_url,
                        response['error']['message']
                        if 'message' in response['error']
                        else ''))
                continue
            if 'labelAnnotations' in response:
                text_response[image_url] = response['labelAnnotations']
            else:
                text_response[image_url] = []
        return text_response
    except errors.HttpError as e:
        print("Http Error for %s: %s" % (image_url, e))
    except KeyError as e2:
        print("Key error: %s" % e2)

In [ ]:
responses = do_batch_request(restaurants_image_lists[0])

In [ ]:
[k, v for k, v in responses if v['description'] in ('meat', 'grillades')]

In [ ]:
[response for response in batch_response if ] 

In [ ]:
[k, v for k, v in batch_response]

In [ ]:
[(label['description'], label['score']) for label in response['responses'][0]['labelAnnotations'] 
 if label['description'] in ('pork belly', 'steak')]

In [ ]:
[label['description'] for label in response['responses'][0]['labelAnnotations']]

In [ ]:
erroraddress ='https://u.tfstatic.com/restaurant_photos/271/254271/169/612/gasterij-de-seterse-hoeve-suggestie-van-de-chef-27f2f.jpg'
reponse = make_request_complete(erroraddress).execute()

In [ ]:
response

In [ ]:
[(label['description'], label['score']) for label in response['responses'][0]['labelAnnotations'] 
 if label['description'] in ('pork belly', 'steak')]

In [ ]:
batch = [burger, steak]

In [ ]:
batch_request
request = collection.annotate(body={'requests' : [
    {
        'image' : {
            'source' : {
                'imageUri' : steak
            }
        },
        'features': [{
                    'type': 'LABEL_DETECTION',
                    'maxResults': 10,
                }]
    }
]})